# Getting started with the Python environment and packages

In [2]:
# import the relevant Python packages

import numpy
import nibabel
import nipype
import matplotlib


ModuleNotFoundError: No module named 'nibabel'

# The PET Brain Imaging Data Structure

## History

The PET modality is a recent addition to BIDS with its introduction via BEP 009. If you're interested in seeing exactly what and how something gets added to BIDS see the pull request for BEP009 [here](https://github.com/bids-standard/bids-specification/pull/633). The results of that extension proposal can be read [here](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/09-positron-emission-tomography.html#positron-emission-tomography) in the bids standard.

# PET data conversion

## PET image data file formats


Before we start to convert data we need to quickly mention that PET image data files come of the scanner in various different formats, some scanners provide DICOM files (.dcm) and others use proprietary formats for example ECAT format (.v) . In order to facilitate easy testing of data conversion across different PET file formats  the [OpenNeuroPET project](https://openneuropet.github.io/) has compiled a bunch of phantom data from different scanner types and is distributing two examples [here](https://drive.google.com/file/d/10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD/view?usp=sharing) . You can download them for testing purposes either manually or in the terminal by typing 

In [ ]:
gdown https://drive.google.com/file/d/10S0H7HAnMmxHNpZLlifR14ykIuiXcBAD/view?usp=sharing --fuzzy

And then unzipping the downloaded file:

In [ ]:
unzip OpenNeuroPET-Demo_raw.zip

You can now look at the file tree:


```
tree OpenNeuroPET-Demo_raw 
OpenNeuroPET-Demo_raw
│   .bidsignore
│   .DS_Store
│   dataset_description.json
│   README
│
├───code
│       .python_conversions.sh.swp
│       matlab_conversions.m
│       python_conversions.sh
│       README.md
│
├───source
│   │   .DS_Store
│   │
│   ├───SiemensBiographPETMR-NRU
│   │       X-CAL_7.PT.Kalibrering_xca.30003.1.2022.04.26.15.04.22.218.14689529.dcm
│   │       X-CAL_7.PT.Kalibrering_xca.30003.10.2022.04.26.15.04.22.218.14689628.dcm
│   │       X-CAL_7.PT.Kalibrering_xca.30003.100.2022.04.26.15.04.22.218.14690618.dcm
│   │       X-CAL_7.PT.Kalibrering_xca.30003.101.2022.04.26.15.04.22.218.14690629.dcm
│   │       ....
│   │
│   └───SiemensHRRT-NRU
│           XCal-Hrrt-2022.04.21.15.43.05_EM_3D.json
│           XCal-Hrrt-2022.04.21.15.43.05_EM_3D.v
│
├───sub-SiemensBiographNRU
│   └───pet
│           sub-SiemensBiographNRU_pet.json
│           sub-SiemensBiographNRU_pet.nii.gz
│
└───sub-SiemensHRRT
    └───pet
            sub-SiemensHRRT_pet.json
            sub-SiemensHRRT_pet.nii.gz
```

Now you have an example dataset where you have source data (both for ECAT and DICOM PET image format) and the PET BIDS data sets constructed for it.

Also if you have access to another PET image file format, or data from a scanner not tested, please reach out to [OpenNeuroPET project](https://openneuropet.github.io/) in order to add a phantom scan in your format.

## Conversion

The [OpenNeuroPET project](https://openneuropet.github.io/) has tried to develop tools for facilitating easy data conversion for PET. The main tool used for this is [PET2BIDS](https://github.com/openneuropet/PET2BIDS) freely available on the [OpenNeuroPET  GitHub repository](https://github.com/openneuropet) along with other resources like altlases or pipelines. It is available for both Python and MatLab. Eventually, [PET2BIDS](https://github.com/openneuropet/PET2BIDS) will also be wrapped inside other BIDS conversion tools such as [BIDScoin](https://github.com/Donders-Institute/bidscoin) or [ezBIDS](https://brainlife.io/ezbids/), but this is work in progress at the moment. 

Besides using  [PET2BIDS](https://github.com/openneuropet/PET2BIDS) there is always the possibility to manually convert a data set to PET BIDS and an example will be shown below. In any case, for dicom data format, one relies on [dcm2niix](https://www.nitrc.org/plugins/mwiki/index.php/dcm2nii:MainPage).

Below we will show two ways of converting your PET data to BIDS: 1) using [PET2BIDS](https://github.com/openneuropet/PET2BIDS) and 2) manually.

### 1) Conversion of PET data using PET2BIDS

Detailed documentation for PET2BIDS can be found [here](https://pet2bids.readthedocs.io/en/latest/index.html#) or on the [Github repo](https://github.com/openneuropet/PET2BIDS/blob/main/README.md).

##### get dcm2niix

Download [dcm2niix](https://www.nitrc.org/plugins/mwiki/index.php/dcm2nii:MainPage), and for Mac and Linux users you can add it to your path, for instance it would could downloaded and unzipped into /bin.

In [ ]:
export PATH="/home/$USER/bin:$PATH" 

##### For Python:

Open your terminal, either regular or anaconda terminal and type. 

In [ ]:
pip install pypet2bids 

Now you already have the converter installed and can go ahead and convert your first dataset! 

In this example, I am converting an image in DICOM format. It should be noted that pypet2bids  contains several different tools and is itself a part of the larger PET library PET2BIDS, the specific tool I will be using for the following DICOM conversion is dcm2niix4pet.

You just need to point dcm2niix4pet to the folder where your data resides, *dcmfolder*, and the folder where you want to output the PET BIDS formatted dataset, *mynewfolder*:

In [ ]:
dcm2niix4pet dcmfolder -d mynewfolder

Note, dcm2niix4pet will do it's best to extract as much information about radiological and blood data from the DICOM files in the dcmfolder. However, dcm2niix4pet can't find information if it isn't there, hence it will often be up to you the user to provide some missing information at the time of conversion. 

Additional information can be provided via the command line with the `--kwargs` argument in the form of key=pair values. For an idea of what this looks like see below:



```bash
dcm2niix4pet /OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR-NRU -d mynewfolder --kwargs TimeZero=ScanStart 
Manufacturer=Siemens 
ManufacturersModelName=Biograph 
InstitutionName="Rigshospitalet, NRU, DK"
BodyPart=Phantom
Units=Bq/mL
TracerName=none
TracerRadionuclide=F18
InjectedRadioactivity=81.24
SpecificRadioactivity=13019.23
ModeOfAdministration=infusion
FrameTimesStart=0
AcquisitionMode="list mode"
ImageDecayCorrected=true
ImageDecayCorrectionTime=0
AttenuationCorrection=MR-corrected
FrameDuration=300
FrameTimesStart=0
```

Now you have a dataset in PET BIDS format. You will probably have gotten some warnings relating to the .json sidecar file. Carefully look at them, since they will help you to catch inconsistencies and missing required fields that you need to add in order for the dataset to pass the BIDS validator as well (see below how that's done). 
You can always edit the .json file, by opening it in a text editor and manually fixing errors. Alternatively, adjust the meta structure you created above to correct the errors.

# PET processing

## PET example data for processing

In the following we will basically follow the tutorial presented in the [PyPetSurfer repository](https://github.com/openneuropet/PET_pipelines/tree/main/pyPetSurfer) and perform a simple data processing of a PET data set. Therefore we need to clone that GitHub repository:


In [ ]:
! git clone https://github.com/openneuropet/PET_pipelines/tree/main/pyPetSurfer .

Then we need to cd to the right directory inside the Github repository:

In [ ]:
 cd pyPetSurfer 

Download this example dataset from OpenNeuro: https://openneuro.org/datasets/ds001421. You can see instructions on how to do this [here](https://openneuro.org/datasets/ds001421/versions/1.2.1/download). In this example, I will use the openneuro cli 

In [ ]:
openneuro download --snapshot 1.2.1 ds001421 ds001421-download/

And now simply perform your first PET data anlysis, including

In [ ]:
%run example.py